# 2.3 Lab: Introduction to ~~R~~ Python

## 2.3.4 Loading Data 

In this lab, we will work through loading external data. Some libraries that will help are: 

* `os` This module provides a portable way of using operating system dependent functionality.  https://docs.python.org/3/library/os.html
* `pathlib` This module offers classes representing filesystem paths with semantics appropriate for different operating systems. https://docs.python.org/3/library/pathlib.html
* `statsmodels` - contains methods for loading R data from R packages. 

### Imports


In [1]:
from typing import List, Dict, Tuple, Sequence, Union # annotation library
import sys # system library 
import os
import pathlib
import tempfile

import statsmodels # root namespace for intellisense and deep reference
import statsmodels.api as statsModels # alias to hide the extra api namespace

import numpy   # as np
import pandas  # as pd

from sklearn import * # import all child namespaces 

import matplotlib.pyplot as pyplot
from mpl_toolkits import mplot3d   # matplotlib 3d plotting
import seaborn 

Loading Data from R Packages

We can use the `statsmodels` package to load data from R libraries. We first call `get_rdataset` method from the `statsmodels.api.datasets` namespace to create a data set reference. Then, we create the `pandas` data frame from the `data` property of the data set. 

Notice that the difference in the `Auto` data frame in the `ISLR` and the `Auto.data` is that the missing value rows have already been removed. The data frame has 392 rows with 9 columns and the original data file has 397 rows. 

**R Code**
```R
# this data is also available in the ISLR package 
# fix function is being used to replace the existing variable assignment

# this is the first load without considering the header row 
# which causes the headers to be row 1
Auto = read.table ("Auto.data")
fix (Auto)

# this is the corrected load with the headers loaded as column names
Auto = read.table ("Auto.data", header = TRUE, na.strings = "?")
fix (Auto)

Auto = read.csv ("Auto.csv", header = TRUE, na.strings = "?")
fix (Auto)
dim (Auto)
Auto [1:4, ]

Auto = na.omit (Auto)
dim (Auto)
names (Auto)
```



In [2]:
# download the R data set as a pandas DataFrame
autoDataSet: statsmodels.datasets.utils.Dataset = statsModels.datasets.get_rdataset ("Auto", "ISLR")

# print out the help information on the data set to get the column documentation
print (autoDataSet.__doc__)

+------+-----------------+
| Auto | R Documentation |
+------+-----------------+

Auto Data Set
-------------

Description
~~~~~~~~~~~

Gas mileage, horsepower, and other information for 392 vehicles.

Usage
~~~~~

::

   Auto

Format
~~~~~~

A data frame with 392 observations on the following 9 variables.

``mpg``
   miles per gallon

``cylinders``
   Number of cylinders between 4 and 8

``displacement``
   Engine displacement (cu. inches)

``horsepower``
   Engine horsepower

``weight``
   Vehicle weight (lbs.)

``acceleration``
   Time to accelerate from 0 to 60 mph (sec.)

``year``
   Model year (modulo 100)

``origin``
   Origin of car (1. American, 2. European, 3. Japanese)

``name``
   Vehicle name

The orginal data contained 408 observations but 16 observations with
missing values were removed.

Source
~~~~~~

This dataset was taken from the StatLib library which is maintained at
Carnegie Mellon University. The dataset was used in the 1983 American
Statistical Association Expos

In [3]:
# Get the pandas DataFrame, use copy so that we can re-run from this point 
# and recover the original data set (non-referenced).
autoDataFrame: pandas.core.frame.DataFrame = autoDataSet.data.copy ()

# preview the data
print ("auto data frame shape: ({0}, {1})".format (autoDataFrame.shape [0], autoDataFrame.shape [1]))
autoDataFrame.head () 

# notice that sets the row index name with the offset of 1 and not 0. 

auto data frame shape: (392, 9)


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
1,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
2,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
3,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
4,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
5,17.0,8,302.0,140,3449,10.5,70,1,ford torino


### Saving and Loading Data from a CSV

A comma-separate values (CSV) file is a common file format for data. It generally has text qualified quotes ("), commas between data columns, and a row terminator that is a carriage return (CR) and line feed (LF). But the extension is commonly used for files that might have other delimitators like tabs or other small changes. It might have headers or not for the column names. 

It is important to understand the file format before importing. You can use a text editor like Notepad++ to preview the file and see the special characters used. 

Reading and writing will be to and from `pandas` data frames, which has helpers to facilitate the operations with the following arguments. 

* `path_or_buf`: str or file handle, default None
* `sep: str`: separator, default is `,`
* `na_rep: str`: character/string for NA values, default is empty string `''`
* `index: bool`: include the index, default is `True` 
* `quotechar: str`: the text quoted characeter, default (`"`)
* `line_terminator`: the line termintor, default is CRLF

More arguments are available to refine the export and import. https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html



In [4]:
# save the auto data frame to a temporary file 
tempFileName: str = os.path.join (tempfile.gettempdir (), "auto.csv")
autoDataFrame.to_csv (path_or_buf = tempFileName, index = False)

# reload the data frame into a second pandas data frame and preview 
autoDataFrame2 = pandas.read_csv (tempFileName)
print ("auto 2 data frame shape: ({0}, {1})".format (autoDataFrame2.shape [0], autoDataFrame2.shape [1]))
autoDataFrame2.head ()

auto 2 data frame shape: (392, 9)


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


### Referencing Locations within the Data Frame

We can reference locations within the data frame using `loc` and `iloc` methods to access groups (series) of data along with the `at` and `iat` methods to access cell values.

`DataFrame.iloc` - access a group of rows and columns by integer position.

`DataFrame.loc` - access a group of rows and columns by label(s).

`DataFrame.iat` - access a single value for a row/column pair by integer position. 

`DataFrame.at`  - access a single value for a row/column pair by label pair. It is important to note that this will use the row name, which might not always start at 0 so that the 1 index is not the same as the row named 1.

In [5]:
# rows 11 and 12, all columns
autoDataFrame.iloc [9:12, :]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
10,15.0,8,390.0,190,3850,8.5,70,1,amc ambassador dpl
11,15.0,8,383.0,170,3563,10.0,70,1,dodge challenger se
12,14.0,8,340.0,160,3609,8.0,70,1,plymouth 'cuda 340


In [6]:
# column year, all rows, head provides first 5 entries 
autoDataFrame.loc [:, "year"].head ()

1    70
2    70
3    70
4    70
5    70
Name: year, dtype: int64

In [7]:
print ("an example of access a row with iat and at")
print ("notice the difference in the row actually returned\n")

# single cell value by row index and column index
print ("accessing with iat index of [10, 3] which is row labeled 11 but index 10 from 0")
print (autoDataFrame.iat [10, 3])

# single cell value by row index and column name
# notice that the first value is the row index name not the 0-offset row index
# this is very important difference.
print ("\naccessing with at index of [10, \"horsepower\"] which is row labeled 10 but index 9")
print (autoDataFrame.at [10, "horsepower"])

# rows 11 and 12, all columns
autoDataFrame.iloc [9:11, :]

an example of access a row with iat and at
notice the difference in the row actually returned

accessing with iat index of [10, 3] which is row labeled 11 but index 10 from 0
170

accessing with at index of [10, "horsepower"] which is row labeled 10 but index 9
190


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
10,15.0,8,390.0,190,3850,8.5,70,1,amc ambassador dpl
11,15.0,8,383.0,170,3563,10.0,70,1,dodge challenger se


### Dealing with Missing Data 

The example from R shows how we use `na.omit (dataFrame)` to exclude rows with missing data. We can use the `dropna` method on the data frame to achieve the same functionality in Python. This method can be used to remove rows that having missing values (`axis = 0`) or columns that have missing values (`axis = 1`). The `how` argument is used to determine if `any` missing values will initiate the drop or if `all` values on that slice must be missing to drop. The `thresh` argument, threshold, provides a threshold that require a minimum number of missing values before drop. The drop is by default no inplace and returned in a new data frame. 

The `pandas` library has a special `pandas.NA` data type for missing and the `numpy` library has `nan` for not a number for numeric columns that do not have valid numeric data. The `nan` cannot be used with integer columns, only with float. The `pandas.NA` can be used with both. The `dropna` function will drop both. 

```Python
# remove missing values. 
DataFrame.dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)[source]
```

```R
# notice the reassignment back to the same variable as the changes are
# not made in place.
Auto = na.omit (Auto)
dim (Auto)
 [1] 392 9

# display the column names in the Auto data frame
names (Auto)
```

In [8]:
# copy the data frame for this example so we can repeat it 
autoDataFrame2 = autoDataFrame.copy ()

# drop any rows that have missing values on any columns
print ("auto data frame shape before drop: ({0}, {1})".format (autoDataFrame2.shape [0], autoDataFrame2.shape [1]))
print (autoDataFrame2.iloc [9:11, 1:7])

# let's remove some values, notice the special `NA` type
autoDataFrame2.at [10, "acceleration"] = numpy.nan
autoDataFrame2.at [11, "horsepower"] = pandas.NA

print ("\nmissing values")
print (autoDataFrame2.iloc [9:11, 1:7])

# drop the rows with missing values 
autoDataFrame2 = autoDataFrame2.dropna (axis = 0, how = "any", inplace = False)
print ("\nauto data frame shape after drop: ({0}, {1})".format (autoDataFrame2.shape [0], autoDataFrame2.shape [1]))

auto data frame shape before drop: (392, 9)
    cylinders  displacement  horsepower  weight  acceleration  year
10          8         390.0         190    3850           8.5    70
11          8         383.0         170    3563          10.0    70

missing values
    cylinders  displacement horsepower  weight  acceleration  year
10          8         390.0        190    3850           NaN    70
11          8         383.0       <NA>    3563          10.0    70

auto data frame shape after drop: (390, 9)


In [9]:
# print column names as list 
autoDataFrame.columns.to_list ()

['mpg',
 'cylinders',
 'displacement',
 'horsepower',
 'weight',
 'acceleration',
 'year',
 'origin',
 'name']